In [65]:
#get data
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
from spotipy import util

#analytics
import pandas as pd

import sys

In [66]:
client_id = '759349ca4662426990ede422e2a1efdb' #Need to create developer profile
client_secret = 'b59771d6de47460babce1ce563df31ed'
username = 'rldilley-us' #Store username
redirect_uri='http://localhost:8080/'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)#Create manager for ease
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Get Data

In [75]:
def get_all_tracks_from_all_user_playlists(username, sp):
    
    playlist_dict = get_all_user_playlists(username,sp)
    
    cols = ['name', 'track_id', 'artists', 'album_name', 'date_added', 'images']
    all_tracks_df = pd.DataFrame( columns = cols)
    
    for name,trackid in playlist_dict.items():
        playlist_tracks_df = get_playlist_tracks_df(trackid, username, sp)
        all_tracks_df = pd.concat([all_tracks_df, playlist_tracks_df])
    
    return all_tracks_df
        
def get_all_user_playlists(username, sp):
    playlists = sp.user_playlists(username)
    playlist_dict = {}
    for playlist in playlists['items']:
        playlist_dict[playlist['name']] = playlist['id']
    
    return playlist_dict

def get_playlist_tracks_df(playlist_id, username, sp):
    
    songs = get_all_tracks_from_api(playlist_id,username, sp)
    
    tracks = {}
    cols = ['name', 'track_id', 'artists', 'album_name', 'date_added', 'images']
    tracks_df = pd.DataFrame( columns = cols)
    for i in range(0, len(songs)):
     if songs[i]['track']['id'] != None: # Removes  local tracks, if any
        #track_ids.append(songs[i]['track']['id'])
        #track_names.append(songs[i]['track']['name'])
        name =  songs[i]['track']['name']
        trackid = songs[i]['track']['id']
        tracks[name] = trackid

        artists = []
        artists_dict = {}
        for x in songs[i]['track']['artists']:
            artists.append(x['name'])
            artists_dict[x['name']] = x['id']

        imgs = []
        for y in songs[i]['track']['album']['images']:
            imgs.append(y['url'])

        tracks_df = tracks_df.append({'name': name, 'track_id': trackid, 'artists': artists, 'album_name': songs[i]['track']['album']['name'], 
                                      'date_added': songs[i]['added_at'], 'images':imgs}
                                       , ignore_index = True)
    return tracks_df

def get_all_tracks_from_api(playlist_id, username, sp):

    playlist = sp.user_playlist_tracks(username,playlist_id)

    #Extract list of songs/tracks
    songs = playlist["items"]
    
    while playlist['next']:
        playlist = sp.next(playlist)
        songs.extend(playlist['items'])
    
    return songs

In [77]:
df = get_all_tracks_from_all_user_playlists(username, sp)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3350 entries, 0 to 206
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        3350 non-null   object
 1   track_id    3350 non-null   object
 2   artists     3350 non-null   object
 3   album_name  3350 non-null   object
 4   date_added  3350 non-null   object
 5   images      3350 non-null   object
dtypes: object(6)
memory usage: 183.2+ KB
